In [8]:
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline

prompt = PromptTemplate.from_template("A {word} is a")

# HuggingFace 파이프라인 생성
hf_pipeline = pipeline(
    "text-generation",
    model="gpt2",
    max_new_tokens=150
)

# LangChain HuggingFacePipeline 생성
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 체인 구성
chain = prompt | llm | StrOutputParser()

# 체인 실행
result = chain.invoke({"word": "tomato"})
print(result)